In [1]:
from mmseg.datasets.viperSeq import ViperSeqDataset
from mmseg.datasets.viper import ViperDataset
from torch.utils.data import DataLoader
from functools import partial
from mmcv.parallel import collate
from mmseg.core.evaluation.metrics import flow_prop_iou

/srv/flash1/skareer6/miniconda3/envs/mmseg/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# dataset settings
dataset_type = 'ViperDataset'
data_root = '/srv/share4/datasets/VIPER/'

#imagenet values
img_norm_cfg = dict(
    mean=[123.675, 116.28, 103.53], std=[58.395, 57.12, 57.375], to_rgb=True)

#crop size from the da-vsn paper code
crop_size = (720, 1280)

In [3]:
# train_pipeline = [
#     dict(type='LoadImageFromFile'),
#     dict(type='LoadAnnotations'),
#     dict(type='Resize', img_scale=(2048, 1024), ratio_range=(0.5, 2.0)),
#     dict(type='RandomCrop', crop_size=crop_size, cat_max_ratio=0.75),
#     dict(type='RandomFlip', prob=0.5),
#     dict(type='PhotoMetricDistortion'),
#     dict(type='Normalize', **img_norm_cfg),
#     dict(type='Pad', size=crop_size, pad_val=0, seg_pad_val=255),
#     dict(type='DefaultFormatBundle'),
#     dict(type='Collect', keys=['img', 'gt_semantic_seg']),
# ]
test_pipeline = {
    "im_load_pipeline": [
        dict(type='LoadImageFromFile'),
        dict(type='LoadAnnotations'),
    ],
    "load_no_ann_pipeline": [
        dict(type='LoadImageFromFile'),
    ],
    "load_flow_pipeline": [
        dict(type='LoadFlowFromFile'),
    ],
    "shared_pipeline": [
        # dict(type='Resize', keep_ratio=True, img_scale=(1080, 1920)),
        # dict(type='RandomCrop', crop_size=crop_size, cat_max_ratio=0.75),
        dict(type='RandomFlip', prob=0.0),
    ],
    "im_pipeline": [
        # dict(type='PhotoMetricDistortion'),
        dict(type='Normalize', **img_norm_cfg),
        # dict(type='Pad', size=crop_size, pad_val=0, seg_pad_val=255),
        # dict(type='DefaultFormatBundle'),
        dict(type='ImageToTensor', keys=['img']),
        dict(type='Collect', keys=['img', 'gt_semantic_seg'])#, meta_keys=[]),
    ],
    "flow_pipeline": [
        dict(type='Pad', size=crop_size, pad_val=0, seg_pad_val=255),
        dict(type='DefaultFormatBundle'), #I don't know what this is
        dict(type='Collect', keys=['img', 'gt_semantic_seg'])#, meta_keys=[]),
    ]
}

img_dir = '/srv/share4/datasets/VIPER/'

In [5]:
viper = ViperSeqDataset(
        data_root=data_root,
        img_dir='val/img',
        ann_dir='val/cls',
        split='splits/val.txt',
        pipeline=test_pipeline,
        frame_offset=1,
        flow_dir="/srv/share4/datasets/VIPER_Flowv3/val/flow_occ"
)

2022-11-15 11:15:49,575 - mmseg - INFO - Loaded 4959 images
2022-11-15 11:15:49,646 - mmseg - INFO - Loaded 4959 images
2022-11-15 11:15:49,654 - mmseg - INFO - Loaded 4959 images


PL:  {'im_load_pipeline': [{'type': 'LoadImageFromFile'}, {'type': 'LoadAnnotations'}], 'load_no_ann_pipeline': [{'type': 'LoadImageFromFile'}], 'load_flow_pipeline': [{'type': 'LoadFlowFromFile'}], 'shared_pipeline': [{'type': 'RandomFlip', 'prob': 0.0}], 'im_pipeline': [{'type': 'Normalize', 'mean': [123.675, 116.28, 103.53], 'std': [58.395, 57.12, 57.375], 'to_rgb': True}, {'type': 'ImageToTensor', 'keys': ['img']}, {'type': 'Collect', 'keys': ['img', 'gt_semantic_seg']}], 'flow_pipeline': [{'type': 'Pad', 'size': (720, 1280), 'pad_val': 0, 'seg_pad_val': 255}, {'type': 'DefaultFormatBundle'}, {'type': 'Collect', 'keys': ['img', 'gt_semantic_seg']}]}
type PL:  <class 'dict'>
type PL:  False


In [6]:
data_loader = DataLoader(
    viper,pdb
    collate_fn=partial(collate, samples_per_gpu=1)
)
print("made dl")
for i, data in enumerate(data_loader):
    print(data)


made dl
{'img_metas': [DataContainer([[{'filename': '/srv/share4/datasets/VIPER/val/img/001/001_00010.jpg', 'ori_filename': '001/001_00010.jpg', 'img_shape': (1080, 1920, 3), 'ori_shape': (1080, 1920, 3), 'pad_shape': (1080, 1920, 3), 'scale_factor': 1.0, 'img_norm_cfg': {'mean': array([123.675, 116.28 , 103.53 ], dtype=float32), 'std': array([58.395, 57.12 , 57.375], dtype=float32), 'to_rgb': True}, 'flip': False, 'flip_direction': 'horizontal'}]])], 'img': [tensor([[[[-1.4500, -1.4500, -1.4329,  ...,  1.4612,  1.4612,  1.4612],
          [-1.3302, -1.3473, -1.3644,  ...,  1.4612,  1.4612,  1.4612],
          [-1.1932, -1.2103, -1.2445,  ...,  1.4612,  1.4612,  1.4612],
          ...,
          [ 1.3242,  1.3584,  1.4098,  ...,  2.0948,  2.0948,  2.0948],
          [ 1.3413,  1.3584,  1.4098,  ...,  2.0948,  2.0948,  2.0948],
          [ 1.3413,  1.3584,  1.3927,  ...,  2.0948,  2.0948,  2.0948]],

         [[-1.3354, -1.3354, -1.3179,  ...,  1.7458,  1.7458,  1.7458],
          [-1.2

KeyboardInterrupt: 

## Regular Viper

In [7]:
# dataset settings
dataset_type = 'ViperDataset'
data_root = '/srv/share4/datasets/VIPER/'

#imagenet values
img_norm_cfg = dict(
    mean=[123.675, 116.28, 103.53], std=[58.395, 57.12, 57.375], to_rgb=True)

#crop size from the da-vsn paper code
crop_size = (720, 1280)
train_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='LoadAnnotations'),
    dict(type='Resize', img_scale=(2048, 1024), ratio_range=(0.5, 2.0)),
    dict(type='RandomCrop', crop_size=crop_size, cat_max_ratio=0.75),
    dict(type='RandomFlip', prob=0.5),
    dict(type='PhotoMetricDistortion'),
    dict(type='Normalize', **img_norm_cfg),
    dict(type='Pad', size=crop_size, pad_val=0, seg_pad_val=255),
    dict(type='DefaultFormatBundle'),
    dict(type='Collect', keys=['img', 'gt_semantic_seg']),
]
test_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(
        type='MultiScaleFlipAug',
        img_scale=(2048, 1024),
        # img_ratios=[0.5, 0.75, 1.0, 1.25, 1.5, 1.75],
        flip=False,
        transforms=[
            # dict(type='Resize', keep_ratio=True),
            # dict(type='RandomFlip'),
            dict(type='Normalize', **img_norm_cfg),
            dict(type='ImageToTensor', keys=['img']),
            dict(type='Collect', keys=['img']),
        ])
]

viper = ViperDataset(
        data_root=data_root,
        img_dir='val/img',
        ann_dir='val/cls',
        split='splits/val.txt',
        pipeline=train_pipeline)

2022-11-15 10:44:20,379 - mmseg - INFO - Loaded 4959 images


PL:  [{'type': 'LoadImageFromFile'}, {'type': 'LoadAnnotations'}, {'type': 'Resize', 'img_scale': (2048, 1024), 'ratio_range': (0.5, 2.0)}, {'type': 'RandomCrop', 'crop_size': (720, 1280), 'cat_max_ratio': 0.75}, {'type': 'RandomFlip', 'prob': 0.5}, {'type': 'PhotoMetricDistortion'}, {'type': 'Normalize', 'mean': [123.675, 116.28, 103.53], 'std': [58.395, 57.12, 57.375], 'to_rgb': True}, {'type': 'Pad', 'size': (720, 1280), 'pad_val': 0, 'seg_pad_val': 255}, {'type': 'DefaultFormatBundle'}, {'type': 'Collect', 'keys': ['img', 'gt_semantic_seg']}]
type PL:  <class 'list'>
type PL:  False


In [20]:
data_loader = DataLoader(
    viper,
    batch_size=2,
    collate_fn=partial(collate, samples_per_gpu=1)
)
print("made dl")
for i, data in enumerate(data_loader):
    # for k, v in data.items():
    print(data)
    print(data["gt_semantic_seg"].data[1].shape)
    print(data["img"].data[0].shape)
    break


made dl
{'img_metas': DataContainer([[{'filename': '/srv/share4/datasets/VIPER/val/img/001/001_00010.jpg', 'ori_filename': '001/001_00010.jpg', 'ori_shape': (1080, 1920, 3), 'img_shape': (720, 1280, 3), 'pad_shape': (720, 1280, 3), 'scale_factor': array([1.1109375, 1.1111112, 1.1109375, 1.1111112], dtype=float32), 'flip': False, 'flip_direction': 'horizontal', 'img_norm_cfg': {'mean': array([123.675, 116.28 , 103.53 ], dtype=float32), 'std': array([58.395, 57.12 , 57.375], dtype=float32), 'to_rgb': True}}], [{'filename': '/srv/share4/datasets/VIPER/val/img/001/001_00020.jpg', 'ori_filename': '001/001_00020.jpg', 'ori_shape': (1080, 1920, 3), 'img_shape': (720, 1280, 3), 'pad_shape': (720, 1280, 3), 'scale_factor': array([0.85572916, 0.85555553, 0.85572916, 0.85555553], dtype=float32), 'flip': True, 'flip_direction': 'horizontal', 'img_norm_cfg': {'mean': array([123.675, 116.28 , 103.53 ], dtype=float32), 'std': array([58.395, 57.12 , 57.375], dtype=float32), 'to_rgb': True}}]]), 'img':